In [ ]:
%cd ..
%pwd

In [ ]:
from experiments.modeleval_utils import *

In [ ]:
def print_classification_report_at_threshold(clf, X, y, threshold):
    y_score = get_y_score(clf, X)
    y_pred = y_score >= threshold
    print(f'{threshold=}')
    report = metrics.classification_report(y, y_pred, digits=4)
    print(report)
    roc_auc = metrics.roc_auc_score(y, y_score)
    average_precision = metrics.average_precision_score(y, y_score)
    print(f'{roc_auc=} {average_precision=}')

In [ ]:
def fit_best_model(output_dir, model, data, feature_type, target, scoring, X_train, y_train):
    pipeline, best_params, best_result = get_pipeline(output_dir, data, feature_type, target, scoring, model)
    pipeline.fit(X_train, y_train)
    return pipeline

### Predict Performance on Regression Model

In [ ]:
output_dir = 'experiments/results_FS'
drop_columns = True

In [ ]:
data = 'bugbug_buglevel'
scoring = 'average_precision'

In [ ]:
kind = 'perf_on_reg' #reg_on_perf, perf_on_reg

feature_type = 'traditional'
print(feature_type)

X, y, features = data_map[feature_type][data]('regression', drop_columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

reg_model = 'rf'
reg_pipeline = fit_best_model(output_dir, reg_model, data, feature_type, 'regression', scoring, X_train, y_train)

X_perf, y_perf, features = data_map[feature_type][data]('performance', drop_columns)
X_perf_train, X_perf_test, y_perf_train, y_perf_test = train_test_split(X_perf, y_perf, test_size=0.1, shuffle=False)

perf_model = 'lr' # best performance model
perf_pipeline = fit_best_model(output_dir, perf_model, data, feature_type, 'performance', scoring, X_perf_train, y_perf_train)

if kind == 'reg_on_perf':
    y_ = y_perf_test
else: # perf_on_reg
    y_ = y_test

roc_fig, roc_ax, pr_fig, pr_ax = plot_roc_auc_rec_prec_for_all_models('', data, feature_type, scoring,
    [('Trained on perf. regs - traditional', perf_pipeline), ('Trained on all regs - traditional', reg_pipeline)],
    X_train, X_test, y_train, y_, save=False, show=False, figsize=(6,4), ylim=[0,1.],
    colors=['tab:blue','tab:green'], linestyles=['solid', 'solid'])


print('Regression Model')
reg_t, _ = get_best_f1_threshold(reg_pipeline, X_train, y_train)
print_classification_report_at_threshold(reg_pipeline, X_test, y_, reg_t)

print('Performance Model')
perf_t, _ = get_best_f1_threshold(perf_pipeline, X_perf_train, y_perf_train)
print_classification_report_at_threshold(perf_pipeline, X_test, y_, perf_t)




feature_type = 'bow'
print(feature_type)

X, y, features = data_map[feature_type][data]('regression')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

reg_model = 'xgb'
reg_pipeline = fit_best_model(output_dir, reg_model, data, feature_type, 'regression', scoring, X_train, y_train)

X_perf, y_perf, features = data_map[feature_type][data]('performance')
X_perf_train, X_perf_test, y_perf_train, y_perf_test = train_test_split(X_perf, y_perf, test_size=0.1, shuffle=False)

perf_model = 'lr' # best performance model
perf_pipeline = fit_best_model(output_dir, perf_model, data, feature_type, 'performance', scoring, X_perf_train, y_perf_train)

_ = plot_roc_auc_rec_prec_for_all_models('', data, feature_type, scoring,
    [('Trained on perf. regs - bow', perf_pipeline), ('Trained on all regs - bow', reg_pipeline)],
    X_train, X_test, y_train, y_, figsize=(6,4), ylim=[0,1.],
    colors=['tab:orange','tab:red'], linestyles=['solid', 'solid'],
    roc_fig=roc_fig, roc_ax=roc_ax, pr_fig=pr_fig, pr_ax=pr_ax,
    save=True, path=os.path.join(output_dir, f'plots_perf_vs_reg/{kind}'))

plt.show()


print('Regression Model')
reg_t, _ = get_best_f1_threshold(reg_pipeline, X_train, y_train)
print_classification_report_at_threshold(reg_pipeline, X_test, y_, reg_t)

print('Performance Model')
perf_t, _ = get_best_f1_threshold(perf_pipeline, X_perf_train, y_perf_train)
print_classification_report_at_threshold(perf_pipeline, X_test, y_, perf_t)
